In [ ]:
# %load init.py
import os
import pickle
import sys
# Enable module import from the parent directory from notebooks
sys.path.append(os.path.abspath('..'))
import time

import matplotlib as mpl
# Select plotting backend
mpl.use('nbAgg')

import matplotlib.pyplot as plt
# Customize plotting
plt.style.use('seaborn-paper')
plt.rcParams['axes.labelsize'] = 11.0
plt.rcParams['axes.titlesize'] = 12.0
plt.rcParams['errorbar.capsize'] = 3.0
plt.rcParams['figure.dpi'] = 72.0
plt.rcParams['figure.titlesize'] = 12.0
plt.rcParams['legend.fontsize'] = 10.
plt.rcParams['lines.linewidth'] = 1.
plt.rcParams['xtick.labelsize'] = 11.0
plt.rcParams['ytick.labelsize'] = 11.0

import numpy as np
import sympy as sp
sp.init_printing(euler=True, use_latex=True)

from IPython import display
from scipy import io, optimize
from sklearn import metrics

import core
import dynamicals
import kernels
import numericals
import utils

In [ ]:
dynamical = dynamicals.ProteinSignallingTransductionWithoutKm() 

# Example
Run the inference algorithm

In [ ]:
config = core.Config()
config.create_time(0, 100, 20, 0, 100, 20, 0, 100, 20)
t_indices = np.array([0, 1, 2, 4, 5, 7, 10, 15, 20, 30, 40, 50, 60, 80, 100]) * config.spl_freq
t_indices = np.array([0, 1, 2, 4, 5, 7, 10, 15, 20, 30, 40, 50, 60, 80, 100]) * config.spl_freq
config.obs_tps = config.obs_tps[t_indices]
config.obs_t_indices = config.obs_t_indices[t_indices]
config.est_tps = config.est_tps[t_indices]
config.est_t_indices = config.est_t_indices[t_indices]

config.X_0 = np.array([1., 0., 1., 0., 0.]) 
config.theta = np.array([0.07, 0.6, 0.05, 0.3, 0.017]) 
config.rho_2 = None

config.phi = [
    # (Kernal name, Kernal parameters)
    ('sigmoid', np.array([1., .4, 16.])),
    ('sigmoid', np.array([.44, .2, 8.])),
    ('sigmoid', np.array([.83, 3, 3.1])),
    ('sigmoid', np.array([.62, 1., 1.1])),
    ('sigmoid', np.array([.83, 2.9, 3])),
]
config.sigma_2 = np.full(dynamical.num_x, 1e-2) 
config.delta = np.full(dynamical.num_x, True)
config.gamma = np.array([1e-3, 1e-3, 1e-5, 1e-5, 5e-3])

config.opt_method = 'Newton-CG'
config.opt_tol = 1e-6
config.max_init_iter = None
config.max_iter = 2000

config.plotting_enabled = True
config.plotting_freq = 1

config.debug = True

config.spl_X = dynamical.generate_sample_path(config.theta, config.rho_2, config.X_0, config.spl_tps)
config.obs_Y = utils.collect_observations(config.spl_X, config.obs_t_indices, config.sigma_2)

gp = core.GaussianProcessRegression(dynamical, config)
gp.run()

lpmf = core.LaplaceMeanFieldSymbolic(dynamical, config, gp)    
lpmf.construct_theta_objective_functions(False, True)
lpmf.construct_X_objective_functions(False, True)
lpmf.run()

# Experiments
Setup for the experiment

In [ ]:
num_repetitions = 10
directory = '../data/ode-protein-without-km/{}/'
config_filename = utils.CONFIG_FILENAME
data_filename = utils.DATA_FILENAME
plotting_config = {
    'x': {
        'xlim': (0, 100),
        'ylim': (0., 1.2)
    }
}

Run experiments for positivity constraint on theta

In [ ]:
for repetition in range(1, num_repetitions + 1):
    print('Starting repetition {}'.format(repetition))
    config = core.Config()
    config.load_config(directory.format(repetition), config_filename)
    gp = core.GaussianProcessRegression(dynamical, config)
    gp.run()
    lpmf = core.LaplaceMeanFieldSymbolic(dynamical, config, gp)    
    lpmf.construct_theta_objective_functions(False, True)
    lpmf.construct_X_objective_functions(False, True)
    lpmf.run()

# Analysis
Load data for plotting

In [ ]:
data = []
for repetition in range(1, num_repetitions + 1):
    tmp = utils.load_data(directory.format(repetition), data_filename.format(repetition))
    data.append(tmp)
    
config = core.Config()
config.load_config(directory.format(repetition), config_filename) 

Plot the state estimation result and the RMSE

In [ ]:
X_mean = utils.get_X_mean(data)
X_var = utils.get_X_var(data)

figure = plt.figure(figsize=(12, 6))
for i in range(dynamical.num_x):
    ax = figure.add_subplot(2, 3, i + 1)
    ax.plot(config.spl_tps, config.spl_X[i], color='C0', linewidth=1.5, label='Path')
    if config.delta[i]:
        ax.scatter(config.obs_tps, config.obs_Y[i], color='C1', marker='x', label='Observation')
    ax.errorbar(config.est_tps, X_mean[i], color='C2', linestyle='--', linewidth=1.5,
                yerr=np.sqrt(X_var[i]), ecolor='0', elinewidth=1., capsize=3., capthick=.5, label='Estimation')    
    ax.set_ylabel(dynamical.x_labels[i])
    ax.set_xlabel('Time')
    handles, labels = ax.get_legend_handles_labels()
    ax.legend(handles=handles, labels=labels, loc=0) 
    ax.set_ylim([0, 1.1])  
    
boxprops = dict(linestyle='-', linewidth=1., color='0')
medianprops = dict(linestyle='-', linewidth=1.2, color='red')
meanpointprops = dict(marker='D', markersize=6., markeredgecolor='green', markerfacecolor='green')

rmse = []
for i in range(5):
    rmse_i = []    
    for j in range(num_repetitions):                   
        rmse_i.append(metrics.mean_squared_error(data[j]['eta_X'][i], config.spl_X[i][config.est_t_indices]))
    rmse_i = np.sqrt(rmse_i)
    rmse.append(rmse_i)
        
ax = figure.add_subplot(2, 3, 6)
ax.boxplot(rmse, labels=dynamical.x_labels, notch=False, showfliers=False, showmeans=True, 
           boxprops=boxprops, medianprops=medianprops, meanprops=meanpointprops, whis=[5, 95])  
ax.set_xlabel('State')
ax.set_ylabel('RMSE')


figure.tight_layout()
plt.show()    

figure.savefig('protein-states-without-km.eps', format='eps', dpi=1000, bbox_inches='tight')

Plot parameter estimation result

In [ ]:
parameters = []
parameters_true = []
for i in range(5):
    parameters_i = []    
    for j in range(num_repetitions):                   
        parameters_i.append(data[j]['eta_theta'][i])
    parameters.append(parameters_i)    
    

figure = plt.figure(figsize=plt.figaspect(1))
ax = figure.add_subplot(1, 1, 1)
labels = ['$' + label + '$' for label in dynamical.theta_labels]
ax.boxplot(parameters, labels=labels, notch=False, showfliers=False, showmeans=True, 
           boxprops=boxprops, medianprops=medianprops, meanprops=meanpointprops, whis=[5, 95])        
ax.set_xlabel('Parameter')
j = 0
for i in ax.get_xticks():
    ax.scatter(i, config.theta[j], color='C0', marker='x')
    j += 1


figure.tight_layout()
plt.show()    

figure.savefig('protein-parameters-without-km.eps', format='eps', dpi=1000, bbox_inches='tight')